In [13]:
import pandas as pd
import json
import os
from pathlib import Path 
from urllib.parse import urlparse
from src.utils.text_utils import extract_post_id, extract_post_id_atom, simple_text_to_id


In [3]:
# Import all JSON files from data/archive directory
import glob

# Get all JSON files in the archive directory
json_files = glob.glob('data/archive/*.json')
print(f"Found {len(json_files)} JSON files:")
for file in json_files:
    print(f"  - {file}")

# Load all JSON files into a dictionary
all_data = {}
for file_path in json_files:
    # Extract filename without path and extension for the key
    file_name = os.path.basename(file_path).replace('.json', '')
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            all_data[file_name] = pd.DataFrame(json.load(f))
        print(f"✓ Successfully loaded {file_name}")
    except Exception as e:
        print(f"✗ Error loading {file_name}: {str(e)}")

print(f"\nLoaded data keys: {list(all_data.keys())}")

# Display basic info about each file
for key, data in all_data.items():
    if isinstance(data, list):
        print(f"{key}: {len(data)} items")
    elif isinstance(data, dict):
        print(f"{key}: dictionary with {len(data)} keys")
    else:
        print(f"{key}: {type(data)}")


Found 5 JSON files:
  - data/archive\events.json
  - data/archive\events_1.json
  - data/archive\events_2.json
  - data/archive\events_3.json
  - data/archive\events_4.json
✓ Successfully loaded events
✓ Successfully loaded events_1
✓ Successfully loaded events_2
✓ Successfully loaded events_3
✓ Successfully loaded events_4

Loaded data keys: ['events', 'events_1', 'events_2', 'events_3', 'events_4']
events: <class 'pandas.core.frame.DataFrame'>
events_1: <class 'pandas.core.frame.DataFrame'>
events_2: <class 'pandas.core.frame.DataFrame'>
events_3: <class 'pandas.core.frame.DataFrame'>
events_4: <class 'pandas.core.frame.DataFrame'>


In [33]:

all_data['events_3'].columns

Index(['title', 'blurb', 'description', 'guid', 'url', 'price_display',
       'price', 'is_free', 'organiser', 'age_group_display', 'min_age',
       'max_age', 'datetime_display', 'start_datetime', 'end_datetime',
       'venue_name', 'categories', 'scraped_on', 'full_address', 'latitude',
       'longitude', 'images', 'id'],
      dtype='object')

In [35]:
_df = pd.DataFrame(columns=['blog_name', 'post_id', 'timestamp'])

for key,df in all_data.items():
    df.rename(columns={"index":"id"}, inplace=True)
    df['id'] = df['id'].apply(lambda x: str(int(x)))

    for idx, row in df.iterrows():
        row['id'] = str(int(row['id']))
        blogname = urlparse(row['guid']).netloc.replace(".com","").replace("https://", "")
        if "." in blogname:
            blogname = blogname.split(".")[1]

        post_id = ''
        if blogname == "bykido":
            post_id = extract_post_id_atom(row['guid'])
        else:
            post_id = extract_post_id(row['guid'])
        
        _df.loc[len(_df)] = [blogname, post_id, row['scraped_on']]

_df


,blog_name,post_id,timestamp
0,honeykidsasia,225448,20250612_131316
1,honeykidsasia,225466,20250612_131316
2,honeykidsasia,215974,20250612_131316
3,honeykidsasia,188664,20250612_131316
4,honeykidsasia,188664,20250612_131316
...,...,...,...
1156,thesmartlocal,364340,20250701_131833
1157,thesmartlocal,364340,20250701_131833
1158,thesmartlocal,364340,20250701_131833
1159,thesmartlocal,364340,20250701_131833


In [36]:
for filename, df in all_data.items():
    json_string = df.to_json(orient = 'records', indent=2)
    with open(f"data/archive/{filename}.json","w", encoding = 'utf-8') as f:
        json.dump(json.loads(json_string), f, indent =2, ensure_ascii=False)   

In [37]:

# Group by post_id and create aggregated data
grouped_df = _df.groupby('post_id').agg({
    'blog_name': 'first',
    'timestamp': 'first'
}).reset_index()

# Add the count of events
grouped_df['num_events'] = _df.groupby('post_id').size().values

print(grouped_df)

    post_id              blog_name        timestamp  num_events
0     -2025  thehappymusicfestival  20250625_103253           1
1     10158                 bykido  20250625_103253           1
2     10166            skoolopedia  20250625_103253           6
3     10227            skoolopedia  20250625_103253           6
4    103119            sassymamasg  20250613_100921           5
..      ...                    ...              ...         ...
260  970183         theasianparent  20250701_131833           6
261  970307         theasianparent  20250701_131833           3
262  970313         theasianparent  20250701_131833           2
263    9838                 bykido  20250625_103253           2
264    9867          honeykidsasia  20250701_131833          20

[265 rows x 4 columns]


In [39]:
from src.core.database import init_db, execute_query

init_db()

for idx, row in grouped_df.iterrows():
    query = """
        INSERT OR REPLACE INTO processed_articles (blog_name, post_id, timestamp, num_events) 
        VALUES (?, ?, ?, ?)
    """
    execute_query(query, (row['blog_name'], row['post_id'], row['timestamp'], row['num_events']))